## **Enviroment Configuration**
Imports all required libraries

In [8]:
import os
import io
import glob
import random
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw

## **Directory Configuration**
Sets paths / global options. Adjust BASE_DIR, INPUT_DIR, and PREPARED_DIR to match your project.

In [9]:
# -------------------
# Global seed for reproducibility
# -------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
print("Using SEED =", SEED)

# -------------------
# Project paths
# -------------------
BASE_DIR = Path(".").resolve()
INPUT_DIR = BASE_DIR / "input"          # where parquet files live
PREPARED_DIR = BASE_DIR / "prepared"    # where we will write images

# Output folder structure (single copy of clean images)
subdirs = [
    PREPARED_DIR / "original",

    PREPARED_DIR / "denoise" / "input",
    PREPARED_DIR / "superres" / "input",
    PREPARED_DIR / "colorization" / "input",

    PREPARED_DIR / "inpainting" / "input",
    PREPARED_DIR / "inpainting" / "mask",
]

for d in subdirs:
    d.mkdir(parents=True, exist_ok=True)

print("Base dir    :", BASE_DIR)
print("Input dir   :", INPUT_DIR)
print("Prepared dir:", PREPARED_DIR)

Using SEED = 42
Base dir    : D:\assignments\Assignments\AAI521\FinalProject\Computer_Vision_Project
Input dir   : D:\assignments\Assignments\AAI521\FinalProject\Computer_Vision_Project\input
Prepared dir: D:\assignments\Assignments\AAI521\FinalProject\Computer_Vision_Project\prepared


## **Image corruption helpers (noise, low-res, grayscale, inpainting)**

Defines functions to:

* Add Gaussian noise (denoising task)

* Make a low-resolution version (super-resolution task)

* Convert to grayscale (colorization task)

* Apply random white rectangles + generate a binary mask (inpainting)

In [10]:
# ============== helpers for image ops ==============
def save_image(img: Image.Image, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    img.save(path)


def add_gaussian_noise(img: Image.Image, sigma: float = 0.1) -> Image.Image:
    """Add Gaussian noise in [0,1] range with std=sigma."""
    arr = np.array(img).astype(np.float32) / 255.0
    noise = np.random.normal(0.0, sigma, arr.shape).astype(np.float32)# ----- Denoising: add Gaussian noise -----
def add_gaussian_noise(img: Image.Image, sigma: float = 25.0) -> Image.Image:
    arr = np.array(img).astype(np.float32)
    noise = np.random.normal(0.0, sigma, arr.shape)
    noisy = np.clip(arr + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy)


# ----- Super-resolution: downsample then upscale -----
def make_low_res(img: Image.Image, factor: int = 4) -> Image.Image:
    w, h = img.size
    small = img.resize((max(1, w // factor), max(1, h // factor)),
                       resample=Image.BICUBIC)
    back = small.resize((w, h), resample=Image.BICUBIC)
    return back


# ----- Colorization: grayscale as 3-channel RGB -----
def make_grayscale_rgb(img: Image.Image) -> Image.Image:
    return img.convert("L").convert("RGB")


# ----- Inpainting: random white rectangles + binary mask -----
def apply_random_blocks_for_inpainting(
    img: Image.Image,
    num_blocks: int = 3,
    min_frac: float = 0.05,
    max_frac: float = 0.15,
) -> tuple[Image.Image, Image.Image]:
    """
    Draw random white rectangles on the image (damaged),
    and create a binary mask (L) where white = region to inpaint.
    """
    w, h = img.size
    damaged = img.copy()
    mask = Image.new("L", (w, h), 0)  # 0 keep, 255 inpaint

    draw_dmg = ImageDraw.Draw(damaged)
    draw_mask = ImageDraw.Draw(mask)

    for _ in range(num_blocks):
        bw = int(random.uniform(min_frac, max_frac) * w)
        bh = int(random.uniform(min_frac, max_frac) * h)
        x0 = random.randint(0, max(0, w - bw))
        y0 = random.randint(0, max(0, h - bh))
        x1, y1 = x0 + bw, y0 + bh

        draw_dmg.rectangle([x0, y0, x1, y1], fill=(255, 255, 255))
        draw_mask.rectangle([x0, y0, x1, y1], fill=255)

    return damaged, mask

    noisy = np.clip(arr + noise, 0.0, 1.0)
    noisy_uint8 = (noisy * 255).astype(np.uint8)
    return Image.fromarray(noisy_uint8)


def make_low_resolution(img: Image.Image, scale: int = 4) -> Image.Image:
    """Downsample by 'scale' and upsample back to original size."""
    w, h = img.size
    small = img.resize((max(1, w // scale), max(1, h // scale)), Image.BICUBIC)
    low_res_back = small.resize((w, h), Image.BICUBIC)
    return low_res_back


def create_random_box_mask(
    img: Image.Image,
    num_boxes: int = 3,
    box_size_range=(0.1, 0.4),
) -> Image.Image:
    """
    Create a mask: white (255) where we want to inpaint, black(0) elsewhere.
    """
    w, h = img.size
    mask = np.zeros((h, w), dtype=np.uint8)

    for _ in range(num_boxes):
        box_w = int(random.uniform(*box_size_range) * w)
        box_h = int(random.uniform(*box_size_range) * h)

        x1 = random.randint(0, max(0, w - box_w))
        y1 = random.randint(0, max(0, h - box_h))
        x2 = min(w, x1 + box_w)
        y2 = min(h, y1 + box_h)

        mask[y1:y2, x1:x2] = 255

    return Image.fromarray(mask, mode="L")


def apply_mask_to_image(
    img: Image.Image,
    mask: Image.Image,
    fill_value=(255, 255, 255),
) -> Image.Image:
    """Overwrite masked regions with a solid color to create 'damaged' image."""
    img_arr = np.array(img).copy()
    mask_arr = np.array(mask)

    if mask_arr.ndim == 3:
        mask_arr = mask_arr[..., 0]

    damaged = img_arr.copy()
    damaged[mask_arr == 255] = np.array(fill_value, dtype=np.uint8)
    return Image.fromarray(damaged)


# Convert common parquet schema format to png

Defines load_image_from_row(row) to match your parquet schema.

Supported Schemas:

* image_bytes column (raw bytes)

* image_path column (string path)

* image column that is already PIL or bytes

In [11]:
import io
from PIL import Image

def load_image_from_row(row):
    """
    Convert a parquet row into a PIL.Image.

    This version tries several common patterns:
      1) 'image_bytes' column with raw bytes
      2) 'image_path'  column with a path
      3) 'image' column:
           - dict with 'bytes' and/or 'path'
           - object with .to_pil()
           - raw bytes
    """

    # ---- Case 1: explicit raw bytes column ----
    if "image_bytes" in row and row["image_bytes"] is not None:
        data = row["image_bytes"]
        if isinstance(data, memoryview):
            data = data.tobytes()
        return Image.open(io.BytesIO(data)).convert("RGB")

    # ---- Case 2: explicit path column ----
    if "image_path" in row and row["image_path"] is not None:
        img_path = Path(row["image_path"])
        if not img_path.is_absolute():
            img_path = BASE_DIR / img_path
        return Image.open(img_path).convert("RGB")

    # ---- Case 3: generic 'image' column ----
    if "image" in row and row["image"] is not None:
        val = row["image"]

        # 3a) Already a PIL image
        if isinstance(val, Image.Image):
            return val.convert("RGB")

        # 3b) HF-style dict: {'bytes': ..., 'path': ...}
        if isinstance(val, dict):
            if "bytes" in val and val["bytes"] is not None:
                data = val["bytes"]
                if isinstance(data, memoryview):
                    data = data.tobytes()
                return Image.open(io.BytesIO(data)).convert("RGB")
            if "path" in val and val["path"] is not None:
                img_path = Path(val["path"])
                if not img_path.is_absolute():
                    img_path = BASE_DIR / img_path
                return Image.open(img_path).convert("RGB")

        # 3c) Object with a .to_pil() method (e.g., some dataset image types)
        if hasattr(val, "to_pil"):
            return val.to_pil().convert("RGB")

        # 3d) Raw bytes stored directly
        if isinstance(val, (bytes, bytearray, memoryview)):
            return Image.open(io.BytesIO(bytes(val))).convert("RGB")

    # ---- If you have a differently named column (e.g. 'jpg', 'photo'), add a case here ----
    # Example:
    # if "jpg" in row and row["jpg"] is not None:
    #     data = row["jpg"]
    #     return Image.open(io.BytesIO(data)).convert("RGB")

    raise ValueError(
        "Row does not contain a recognizable image field. "
        "Check df.columns and customize load_image_from_row() to your schema."
    )


# Saving Corrupted Images
Saves one clean image (prepared/original/). Saves only the corrupted versions into each task’s input/ (and mask/ for inpainting).

In [12]:
def save_image_variants_single_target(base_img: Image.Image, base_name: str):
    """
    Given a clean RGB base_img and a base_name (e.g. 'file_000001.png'),
    create and save:

      prepared/original/base_name           → clean image (single copy)

      prepared/denoise/input/base_name      → noisy image
      prepared/superres/input/base_name     → low-res-upscaled image
      prepared/colorization/input/base_name → grayscale RGB image

      prepared/inpainting/input/base_name   → damaged w/ white blocks
      prepared/inpainting/mask/base_name    → binary mask (white=inpaint)

    No per-task 'target' directories; training will use original/ as target.
    """
    # Optionally resize large images to a max side (e.g. 512)
    max_side = 512
    w, h = base_img.size
    scale = min(1.0, max_side / max(w, h))
    if scale < 1.0:
        new_size = (int(w * scale), int(h * scale))
        base_img = base_img.resize(new_size, resample=Image.BICUBIC)

    # 1) Save clean original (single copy)
    orig_path = PREPARED_DIR / "original" / base_name
    orig_path.parent.mkdir(parents=True, exist_ok=True)
    base_img.save(orig_path)

    # 2) Denoise input
    noisy = add_gaussian_noise(base_img, sigma=25.0)
    noisy.save(PREPARED_DIR / "denoise" / "input" / base_name)

    # 3) Super-resolution input
    lowres = make_low_res(base_img, factor=4)
    lowres.save(PREPARED_DIR / "superres" / "input" / base_name)

    # 4) Colorization input
    gray_rgb = make_grayscale_rgb(base_img)
    gray_rgb.save(PREPARED_DIR / "colorization" / "input" / base_name)

    # 5) Inpainting input + mask
    damaged, mask = apply_random_blocks_for_inpainting(base_img, num_blocks=3)
    damaged.save(PREPARED_DIR / "inpainting" / "input" / base_name)
    mask.save(PREPARED_DIR / "inpainting" / "mask" / base_name)


In [13]:
max_rows_per_file = 50  # e.g. 100 to limit, or None for all rows

parquet_files = sorted(INPUT_DIR.glob("*.parquet"))
print("Found parquet files:")
for p in parquet_files:
    print(" -", p.name)

total_saved = 0

for parquet_path in parquet_files:
    print(f"\nProcessing {parquet_path}...")
    try:
        df = pd.read_parquet(parquet_path)
    except Exception as e:
        print(f"  [ERROR] Could not read {parquet_path}: {e}")
        continue

    print("  Rows:", len(df))
    if len(df) == 0:
        continue

    for idx, (_, row) in enumerate(df.iterrows()):
        if max_rows_per_file is not None and idx >= max_rows_per_file:
            break

        try:
            img = load_image_from_row(row)
        except Exception as e:
            print(f"  [WARN] Skipping row {idx}: {e}")
            continue

        if img.mode != "RGB":
            img = img.convert("RGB")

        # unique name: parquetstem_rowindex.png
        base_name = f"{parquet_path.stem}_{idx:06d}.png"

        save_image_variants_single_target(img, base_name)
        total_saved += 1

    print(f"  Done {parquet_path.name}, rows processed: "
          f"{min(len(df), max_rows_per_file or len(df))}")

print("\nAll done.")
print("Total clean images processed:", total_saved)


Found parquet files:
 - test-00000-of-00028.parquet

Processing D:\assignments\Assignments\AAI521\FinalProject\Computer_Vision_Project\input\test-00000-of-00028.parquet...
  Rows: 3572
  Done test-00000-of-00028.parquet, rows processed: 50

All done.
Total clean images processed: 50
